# Use of TF-IDF on large dataset for string matching

## There are many fuzzy matching algorithms that work fine on small dataset. However, they fall short when used on even modest data sets of greater than a few thousand records. This is because they compare each record to all the other records in the data set. Here we will use TF_IDF to compare list of 3000 names in a lookup database of around 1 million names.

In [5]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, xlabel, ylabel
%matplotlib inline
from matplotlib.path import Path
from matplotlib.figure import Figure
from matplotlib.patches import PathPatch
from matplotlib.patches import Patch
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.cm as cm
from IPython.core.display import display
import matplotlib.patches as patches
import math
import numpy as np
from numpy import percentile
from numpy.random import seed
import os
from sklearn.preprocessing import StandardScaler
import datetime as dt
import time
import spacy
import re
import pyodbc
import sqlalchemy as sal
from sklearn.preprocessing import normalize 
from sqlalchemy import create_engine
import pandas as pd
from scipy.stats import bartlett
from scipy.stats import boxcox
from sklearn.preprocessing import normalize 
from scipy.stats import jarque_bera
from scipy.stats import levene
from scipy.stats import normaltest
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from scipy.stats import zscore
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score #
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import precision_recall_curve
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
import xlsxwriter
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, \
    adjusted_mutual_info_score, adjusted_rand_score
from sklearn.metrics import silhouette_samples,  silhouette_score
from sklearn.metrics.cluster import contingency_matrix

In [3]:
# import en_core_web_md
# nlp = spacy.load('en_core_web_md')
from random import sample 

In [6]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_colwidth', -1) 

## Import two data sources

In [7]:
## import the query data
import_path = r'D:\dev_data\re\hcad'
file_name1 = 'pro_names.csv'
file1 = import_path+"\\"+ file_name1
dfp0 = pd.read_csv(file1)
dfp0.shape

FileNotFoundError: [Errno 2] File D:\dev_data\re\hcad\pro_names.csv does not exist: 'D:\\dev_data\\re\\hcad\\pro_names.csv'

In [5]:
dfp0.head(2)

,Unnamed: 0,DOCK_NUM,DE_NAME,DE_FIRST_NAME,DE_MIDDLE_NAME,DE_LAST_NAME,de_prop_given
0,0,471514,CAROLYN A BROWN,CAROLYN,A,BROWN,0
1,1,485685,LUCIO SOLIS,LUCIO,NaN,SOLIS,1


In [6]:
# Import lookup data
owner_cols = ['ACCOUNT', 'MAILTO']
file_name2 = 'ss_owners.csv'
file2 = import_path+"\\"+ file_name2
dfo0 = pd.read_csv(file2,  dtype= str, encoding = "ISO-8859-1", names=owner_cols, skiprows=1)
dfo0.shape

(1087036, 2)

In [7]:
dfo0.head(3)

,ACCOUNT,MAILTO
0,0032180000021,SANTOS DOLORES ST JOHN
1,0032180000022,GRIMALDO ROSIE
2,0032180000023,GARCIA ANTONIO


## Helper functions

In [8]:
# Utility function for standard text cleaning
def text_cleaner(text):
    text = re.sub(r'"','',text)
    text = re.sub(r'&','',text)
    ext = re.sub("[\[].*?[\]];", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

## Preprocess lookup dataset

In [9]:
dfo1 = dfo0.copy()

In [10]:
# Convert column of names to string and clean
l = dfo1['MAILTO'].tolist() 
l=['missing' if x is np.nan else x for x in l]
s = '||||'.join(l).lower()
sc = text_cleaner(s)
#sc[:10]
names = [str(x) for x in sc.split('||||') if x]
names[:3]

['santos dolores st john', 'grimaldo rosie', 'garcia antonio']

In [11]:
dfo2 = pd.DataFrame(names, index=dfo1.index, columns=['MAILTO_cleaned'])
dfo2.head(3)

,MAILTO_cleaned
0,santos dolores st john
1,grimaldo rosie
2,garcia antonio


In [12]:
# Combine original df with cleaned names
dfo3 = pd.concat([dfo1, dfo2], axis=1)
dfo3['source'] = 'owners' # Add this so can identify the array later
dfo3['de_prop_given'] = '' # Placeholder for future use
dfo3.shape

(1087036, 5)

In [13]:
dfo3.head(3)

,ACCOUNT,MAILTO,MAILTO_cleaned,source,de_prop_given
0,0032180000021,SANTOS DOLORES ST JOHN,santos dolores st john,owners,
1,0032180000022,GRIMALDO ROSIE,grimaldo rosie,owners,
2,0032180000023,GARCIA ANTONIO,garcia antonio,owners,


In [14]:
# Remove duplicates
dfo3 = dfo3.drop_duplicates(subset='ACCOUNT', keep="last")
dfo3.shape

(1087036, 5)

In [15]:
# Tag commercial names by regex pattern
def find_pat(text):
    if re.search(r" llc|current owner|  inc| lc| ltd| lp| churchcorp| company|city of houston|\
        county of harris|state of texas| company| harris county|harris county|county of harris|\
            texas department| city of katy|parcel", text):
        return 1
    return   0
# Applu the function
dfo3['non_person'] = dfo3['MAILTO_cleaned'].apply(find_pat)

In [16]:
# Number of commercial entries
dfo3['non_person'].sum()

68266

In [17]:
# Drop commercial entries
dfo3 = dfo3.drop(dfo3[dfo3.non_person ==1].index)
dfo3.shape

(1018770, 6)

In [18]:
# Extract the last name
dfo3['l_name'] = dfo3['MAILTO_cleaned'].str.extract('^([\w\-]+)', expand=True)
dfo3 = dfo3[~dfo3['l_name'].isnull()] # Filter away those names that start with digits
dfo3.sample(3)

,ACCOUNT,MAILTO,MAILTO_cleaned,source,de_prop_given,non_person,l_name
873629,1251620060091,CHEN VAN,chen van,owners,,0,chen
379310,1012910000022,SULLIVAN WILLIAM & BARBARA,sullivan william barbara,owners,,0,sullivan
66471,0440660000276,ZARSKY WESLEY,zarsky wesley,owners,,0,zarsky


In [19]:
dfo3.shape

(1018347, 7)

In [20]:
dfo31 = dfo3.copy()
#dfo31 = dfo3[:10000]
dfo3.shape, dfo31.shape

((1018347, 7), (1018347, 7))

# ignore this section

In [21]:
# # Convert column of names to string and clean
# lst = dfp0['DE_LAST_NAME'].tolist()
# lst_str = '||||'.join(lst).lower()
# p_lnames = [str(x) for x in lst_str.split('||||') if x] 
# p_lnames.sort()

# # Remove duplicates
# res = [] 
# [res.append(x) for x in p_lnames if x not in res] 
# p_lnames = res

In [22]:
# # Create regex mask for last names
# lname_empty = []
# for count, value in enumerate(p_lnames):
#     lname_empty.append("((?:\s|^)"+value+"(?:\s|$))") # Regex get a whole word and not partial in any part

# lname_reg_str = '|'.join(lname_empty)
# lname_pat = re.compile(lname_reg_str) #'^\\b'+reg_str+'\\b')
# lname_reg_str[:100]

### First name regex processing

In [23]:
# # Convert column of names to string and clean
# lst = dfp0['DE_FIRST_NAME'].tolist()
# lst_str = '||||'.join(lst).lower()
# p_fnames = [str(x) for x in lst_str.split('||||') if x] 
# p_fnames.sort()

# # Remove duplicates
# res = [] 
# [res.append(x) for x in p_fnames if x not in res] 
# p_fnames = res

In [24]:
# # Create regex mask for first names
# fname_empty = []
# for count, value in enumerate(p_fnames):
#     fname_empty.append("((?:\s|^)"+value+"(?:\s|$))") # Regex get a whole word and not partial in any part

# fname_reg_str = '|'.join(fname_empty)
# fname_pat = re.compile(fname_reg_str) #'^\\b'+reg_str+'\\b')

## This section is preparation for using with Dask in case need parallelization later

In [25]:
# # Remove null to can save as parquet
# dfo3 = dfo3[dfo3['MAILTO'].notna()]
# dfo3 = dfo3[dfo3['l_name'].notna()]
# dfo3['source'] = 'owners'
# dfo3 = dfo3.astype(str)
# dfo3 = dfo3.reset_index(drop=True)

In [26]:
# # Check to see how frequent last names are
# from collections import Counter
# wrd_lst = dfo31['l_name'].tolist()
# words_to_count = (word for word in wrd_lst)
# c = Counter(words_to_count)
# print (c.most_common(20))

In [27]:
# import dask
# import pyarrow.parquet as pq
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
# %matplotlib inline

# # Save owner data as parquet
# export_path = r'D:\\dev_data\\re\\hcad\\lookup_data\\'
# ddf = dd.from_pandas(dfo31, chunksize=1000)
# save_dir = export_path
# ddf.to_parquet(save_dir)

# Ignore before here

## Preprocess querry dataset

In [28]:
dfp1 = dfp0[['DOCK_NUM', 'DE_NAME', 'de_prop_given']]
dfp1 = dfp1.applymap(lambda s:s.lower() if type(s) == str else s)
dfp1['source'] = 'prob' # Add this so can identify the array later
dfp1 = dfp1.rename(columns={"DE_NAME": "MAILTO_cleaned", "DOCK_NUM": "ACCOUNT"})
# Remove pro duplicates
dfp1 = dfp1.drop_duplicates(subset='MAILTO_cleaned', keep="last")
#dfp1 = dfp1[:1000]

In [29]:
# Extract the last name
dfp1['l_name'] = dfp1['MAILTO_cleaned'].str.extract('([\w\-]+)$', expand=True)
dfp1.sample(5)

,ACCOUNT,MAILTO_cleaned,de_prop_given,source,l_name
1118,490206,maude a jones,0,prob,jones
1660,487813,marguerite baldwin,0,prob,baldwin
1611,487691,mohammad taha,0,prob,taha
1259,489282,claudette whitefield,1,prob,whitefield
2232,488611,donald l connelly,0,prob,connelly


In [30]:
# Drop extra columns
dfo31 = dfo31.drop(['MAILTO', 'non_person'], 1)

In [31]:
dfo31.columns

Index(['ACCOUNT', 'MAILTO_cleaned', 'source', 'de_prop_given', 'l_name'], dtype='object')

In [32]:
dfp1.columns

Index(['ACCOUNT', 'MAILTO_cleaned', 'de_prop_given', 'source', 'l_name'], dtype='object')

In [33]:
# Combine lookup with query dataset
result = dfo31.append(dfp1, sort=False)
result = result.reset_index(drop=True)

In [34]:
result.groupby('source').sample(n=2, random_state=1)

,ACCOUNT,MAILTO_cleaned,source,de_prop_given,l_name
874134,1276880010027,carr kirlice v,owners,,carr
924294,1301260010006,baker kenneth r mary l,owners,,baker
1018582,490461,elizabeth cowper,prob,0,cowper
1018519,490572,george g edwards,prob,0,edwards


In [35]:
# # Add two test rows to the df for future test. One for each group to verify the results
# test_df = pd.DataFrame(columns=result.columns)
# test_df.loc[0] = ['1111111111111', 'amir athari', 'athari', 'prob']
# test_df.loc[1] = ['1111111111111', 'amir athari', 'athari', 'owners']
# # Append all in one big df
# result = result.append(test_df, sort=False)
# result = result.reset_index(drop=True)
# result = result.astype(str)

In [36]:
# Pick indexes of two groups for future slicing
l_index = result[result['source']=='owners'].index
q_index = result[result['source']=='prob'].index

In [37]:
#THIS ASSUME the index is order need correction
# # Lookup explicit index
# lsi = result[result['source']=='owners'].index.values.astype(int)[0]
# lei = result[result['source']=='owners'].index.values.astype(int)[-1]

# # Query explici index
# qsi = result[result['source']=='prob'].index.values.astype(int)[0]
# qei = result[result['source']=='prob'].index.values.astype(int)[-1]


In [38]:
# result_l = result[lsi:lei+1]
# result_q = result[qsi:qei+1]
# result_l.shape, result_q.shape

# Feature Engineering

### Here we use Tf-idf (term frequency–inverse document frequency) by looking at a normalized count where each word count is divided by the number of rows (i.e. document) this word appears in. I chose this method instead of for example bag-of-words becasue we are comparing names where on average every document has 3 words in it. And also I am not looking for similarities but actual exact match.

I use Tf-idf twice, once to eliminate all documents from lookup dataset that do not have the exact last names like in query data. This will improve the performance. In a second round I will then compare the query data with a smaller loopup data using the second round of Tf-idf.

In [39]:
result[result['l_name'].isnull()]

,ACCOUNT,MAILTO_cleaned,source,de_prop_given,l_name


In [40]:
# use 1-1 word ngrams on last names only
vectorizer = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word'
                                       # ,stop_words='english'
                                       ,ngram_range = (1, 1)
                                       #, min_df = 1
                                       , norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=True)#,
                                       
                                      #  max_df=1, max_features=None)
X = vectorizer.fit_transform(result['l_name'])

In [41]:
print(vectorizer.get_feature_names()[:50])

['00i', '1stop', '21st', '26th', '331eg', '396bp', '3c', '3lm', '3zca', '475xj', '4mb', '4mk', '4y', '6g', '88jg', 'a91', 'aaa', 'aad', 'aagaz', 'aah', 'aaker', 'aakerberg', 'aakquanakhann', 'aal', 'aalders', 'aaloori', 'aals', 'aalund', 'aaly', 'aamir', 'aamodt', 'aamoth', 'aana', 'aanderud', 'aanstoos', 'aaqid', 'aardsma', 'aaron', 'aaronfaridi', 'aarons', 'aaronson', 'aarup', 'aasen', 'aaseng', 'aaser', 'aaserud', 'aasgaard', 'aasim', 'aaz', 'aba']


In [42]:
#nd-array info
X.shape, X.ndim, X.size

((1020102, 120327), 2, 1025585)

In [43]:
# Filter away rows where there is no last name from query list 
# Get similarities of lookup and query dataset 
sim1 = X[l_index].dot(X[q_index].transpose())
sim1.shape, sim1.ndim, sim1.size

((1018347, 1755), 2, 1062335)

In [44]:
# Get non zero values' indexes and their values
nonzero_tup = np.stack(np.nonzero(sim1), axis=-1)
# Filter away zeros and return a list of indexs where there was a match with last names only
res_list1 = [x[0] for x in nonzero_tup]
# Convert list to array as it is expensitve to remvoe duplicates in a large list
res_array = np.array(res_list1)
res_uniques = np.unique(res_array) # This is the smaller lookup dataset where there is one exact match of last name for query data
res_uniques.shape

(315440,)

In [45]:
# Drop irrelevant rows from the original 'combined' dataset
result1 = result.iloc[res_uniques]
# Keep only "owner" rows
result1 = result1[result['source']=='owners']
result1.shape

(315440, 5)

In [46]:
result1.groupby('source').sample(n=2, random_state=1)

,ACCOUNT,MAILTO_cleaned,source,de_prop_given,l_name
125493,0651220080105,ramirez maritza p,owners,,ramirez
847090,1262310030043,martinez victor,owners,,martinez


In [47]:
# Prepare TF-IDf for a more focused lookup dataset
# Append query data to the smaller lookup dataset
result2 = result1.append(dfp1, sort=False)
result2 = result2.reset_index(drop=True)
result2.head(3)

,ACCOUNT,MAILTO_cleaned,source,de_prop_given,l_name
0,0032180000023,garcia antonio,owners,,garcia
1,0032180000024,garcia paul,owners,,garcia
2,0032180000027,martinez carlo p graciela guzman,owners,,martinez


In [48]:
result2.tail(3)

,ACCOUNT,MAILTO_cleaned,source,de_prop_given,l_name
317192,489872,lea fradkin,prob,0,fradkin
317193,489877,elizabeth hossan,prob,0,hossan
317194,489879,clarissa metzger,prob,0,metzger


In [50]:
# Inspect the lookup and query part of the data prior to Tf-idf
result2[result2['source']=='owners'].shape[0], result2[result2['source']=='prob'].shape[0], 

(315440, 1755)

In [51]:
# Vectorize names using 1-4 n-grams
vectorizer = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word'
                                       # ,stop_words='english'
                                       ,ngram_range = (1, 4)
                                       #, min_df = 1
                                       , norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=True)#,
                                       
                                      #  max_df=1, max_features=None)
X1 = vectorizer.fit_transform(result2['MAILTO_cleaned'])

In [56]:
print(vectorizer.get_feature_names()[:10])

['11b', '11b mt', '11b mt sinai', '11b mt sinai ln', '1981', '1981 emily', '1981 emily knobloch', '1981 emily knobloch trst', '1st', '1st cont']


In [57]:
#nd-array info
X1.shape, X1.ndim, X1.size

((317195, 492682), 2, 1536493)

In [58]:
# Get index of lookup and query features so can map to original df
l_index = result2[result2['source']=='owners'].index
q_index = result2[result2['source']=='prob'].index
X1[q_index].shape, X1[l_index].shape[0]

((1755, 492682), 315440)

## Use of cosign similarities to find matching names

In [88]:
start = time.time()
dot_prod_lst = []   # Collect the similarities
l_i_lst = []        # Collect the lookup index
q_i_lst = []        # Collect the query index 
tax_lst = []        # Collect the Account numbers from lookup
dock_num_lst = []   # Collect the Account numbers from query
l_name_lst = []     # Collect the index for names from lookup
q_name_lst = []     # Collect the index for names from query
q_target_lst = []   # Collect the index for targe labels
for i in range(0,X1[q_index].shape[0]): # Loop the query list
        dot=X1[l_index].dot(X1[q_index][i].transpose()) # Get dot product of each query row with all lookup
        if dot.max() > 0.35:
            g = np.round(dot.max(), 3)

            ##### Below print out for inspection purposes
            # print('Dot product {}.'.format(g))
            # print('Lookup index i {}.'.format(i))
            # print('----')
            # print('Lookup source: {}.'.format(result2.iloc[l_index[i]][3]))
            # print('Query source: {}.'.format(result2.iloc[q_index[i]][3]))
            # print('----')
            # print('Lookup Tax Num: {}.'.format(result2.iloc[l_index].iloc[dot.argmax()][0]))
            # print('Lookup name: {}.'.format(result2.iloc[l_index].iloc[dot.argmax()][1]))
            # print('Query name: {}.'.format(result2.iloc[q_index[i]][1]))
            # print('Query Docket Num: {}.'.format(result2.iloc[q_index[i]][0]))
            # print('________________')
            ######
            dot_prod_lst.append(g)
            l_i_lst.append(i)
            tax_lst.append(result2.iloc[l_index].iloc[dot.argmax()][0])
            l_name_lst.append(result2.iloc[l_index].iloc[dot.argmax()][1])
            q_name_lst.append(result2.iloc[l_index].iloc[dot.argmax()][3])
            dock_num_lst.append(result2.iloc[q_index[i]][0])
            q_name_lst.append(result2.iloc[q_index[i]][1])
            q_target_lst.append(result2.iloc[q_index[i]][3])
            
end = time.time()
print(end - start)

190.063227891922


In [89]:
result2.columns

Index(['ACCOUNT', 'MAILTO_cleaned', 'source', 'de_prop_given', 'l_name'], dtype='object')

In [105]:
# Collected lists
lst_of_lists = [l_i_lst, q_i_lst, tax_lst, dock_num_lst, l_name_lst, q_name_lst, q_target_lst]
for i in lst_of_lists:
    print(f'length of each collected list: {len(i)}')

length of each collected list: 1322
length of each collected list: 0
length of each collected list: 1322
length of each collected list: 1322
length of each collected list: 1322
length of each collected list: 2644
length of each collected list: 1322


In [90]:
# Helper loop to find the position of headers for building the df
for i in range(0,result.shape[1]):
    print('---------')
    print(f'lookup for column {i}: {result2.iloc[l_index].iloc[dot.argmax()][i]}')
    print(f'query columns {i}: {result2.iloc[q_index[i]][i]}')


---------
lookup for column 0: 0975020000033
query columns 0: 489910
---------
lookup for column 1: metzger clarissa m % clarissa metzger sur trus
query columns 1: gladys godkin
---------
lookup for column 2: owners
query columns 2: prob
---------
lookup for column 3: 
query columns 3: 0
---------
lookup for column 4: metzger
query columns 4: britton


In [91]:
# Prepare a df for all collected data from previous step
sim_df = pd.DataFrame({'lookup_tax_id': tax_lst, 'lookup_name': l_name_lst \
                     , 'query_dock_num': dock_num_lst, 'query_name': q_name_lst \
                     , 'sim_score': dot_prod_lst, 'lookup_index': l_i_lst, 'target_labels': q_target_lst \
                     , }).sort_values(by=['sim_score'], ascending=False)
sim_df.shape

ValueError: arrays must all be same length

In [ ]:
sim_df.head(3)

In [ ]:
lst

In [ ]:
# Create df with vectorized values, headings and original index
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index=result.index)
tfidf_df.reset_index()
tfidf_df.shape

In [ ]:
result.tail()

In [ ]:
result.shape[0]

In [ ]:
d = pd.DataFrame()
for i in range(1,10):
    d = cosine_similarity(tfidf_df, tfidf_df.iloc[i:i+1])
    d.shape
    flat_list = [item for sublist in d for item in sublist]
    best_match_index = np.argmax(flat_list)
    #print(best_match_index)
    best_match_val = max(d)
    if best_match_val.any() > .80:
         print(result.iloc[best_match_index, 1])
    
#df_temp = cosine_similarity(tfidf_df, tfidf_df.iloc[1000:1001]) # Here I assume that the parent vector is stored as the first row in the dataframe, but you could also store it separately

# n = 10 # or however many you want
# n_largest = tfidf_df['dist'].nlargest(n )  # this contains the parent itself as the most similar entry, hence n+1 to get n children

In [ ]:
# DO NOT DELETE
#https://stackoverflow.com/questions/53875473/cosine-similarity-for-very-large-dataset

In [ ]:
tfidf_df.head(2)

In [ ]:

owners_i = result[result['source']=='owners'].index
hcad_df = result[result['source']=='hcad']

In [ ]:
result.head(2)

In [ ]:
# Get owners index
owners_i = result[result['source']=='owners'].index
# Create empty distance df
dist_df = pd.DataFrame(index=result.index)#, columns=owners_i)
# Add other meta data to dist df
dist_df['ACCOUNT'] = result['ACCOUNT']
dist_df['MAILTO_cleaned'] = result['MAILTO_cleaned']
dist_df['source'] = result['source']
# Move col to left
cols = dist_df.columns.tolist()
cols = cols[-3:] + cols[:-3]
dist_df = dist_df[cols]
dist_df.head(2)

In [ ]:
dist_df.shape

In [ ]:
dist_df_temp.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(result2)
similarity.shape

In [ ]:
# Remove query rows
result3 = result2[result2['source']=='hcad']
# Remove extra dict values
result4 = result3.drop(['source', 'MAILTO_cleaned'], axis=1)
# Create dict
hcad_d = result4.set_index('ACCOUNT').T.to_dict('list')
# Show n first key:value
# {k: hcad_d[k] for k in list(hcad_d)[:1]}

In [ ]:
# Remove query rows
result10 = result2[result2['source']=='owners']
# Remove extra dict values
result11 = result10.drop(['source', 'MAILTO_cleaned'], axis=1)
# Create dict
owners_d = result11.set_index('ACCOUNT').T.to_dict('list')
# Show n first key:value
# {k:owners_d[k] for k in list(owners_d)[:1]}

In [ ]:
shared_items = {k: x[k] for k in hcad_d if k in owners_d and hcad_d[k] == owners[k]}
print (len(shared_items))

In [ ]:
#oa = np.array(list(owners_d.items())[1])


In [ ]:
l = dfp1['DOCK_NUM'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
dock_num = [str(x) for x in s.split('||||') if x]
################
l = dfp1['DE_NAME'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
de_name = [str(x) for x in s.split('||||') if x]


In [ ]:
t=[]
# Part 01:
for j,k in enumerate(tfidf_ddj.values):
    for n in range(len(k)):
        t.append([j,n,k[n]])

# Part 02:
qq=[]
for i in range(len(t)):
    if t[i][0]==t[i][1]:
        qq.append([t[i][0],t[i][1],0])
    else:
        qq.append(t[i])
qq[:5]

In [ ]:
from collections import defaultdict
u=defaultdict(list)

# Part 01:
for i in range(len(qq)):
    u[qq[i][0]].append(qq[i][2])
    
updated_df=pd.DataFrame(u)

# Part 02:
position_maxVal=[]
for i in range(len(updated_df)):
    position_maxVal.append(np.argmax(updated_df[i]))

In [ ]:
sent_comp=[]

for j in position_maxVal: # list of highest similarity index positions
    # this creates in order our names w/ highest similiarity by row    
    sent_comp.append(names[j])
    sent_comp

# names based on highest similarity value per row as DF
similar_names=pd.DataFrame(sent_comp,columns=['Similar Names'])

# similiarity values rounded 4 decimal places finding max value per row
similarity_value_=pd.DataFrame(round(updated_df.max(axis=1),4), columns=['Similarity Value'])

In [ ]:
# Get accounts and names
p_accounts=pd.DataFrame(accounts,columns=['Accounts'])
p_names=pd.DataFrame(names,columns=['Names'])
p_dock_num=pd.DataFrame(dock_num,columns=['DockNum'])
p_de_name=pd.DataFrame(de_name,columns=['De_Name'])

# put everything together
cos_sim_df=pd.concat([p_accounts, p_names,similar_names,similarity_value_, p_dock_num, p_de_name], axis=1)

In [ ]:
cos_sim_df.head()

## Work on first names

In [ ]:
p_fnames[:3]

In [ ]:
# Create regext list of first names
empty = []
for count, value in enumerate(p_fnames):
    empty.append("((?:\s|^)"+value+"(?:\s|$))") # Regex get a whole word and not partial in any part

reg_str = '|'.join(empty)
fname_pat = re.compile(reg_str) #'^\\b'+reg_str+'\\b')

In [ ]:
fn_mask = dfo4[['MAILTO_cleaned']].apply(lambda x: x.str.contains(fname_pat, regex=True )).any(axis=1)
dfo3.shape, dfo4.shape, dfo4[fn_mask].shape

In [ ]:
dfo5 = dfo4[fn_mask].copy()
dfo5.shape

In [ ]:
dfo5.columns

In [ ]:
# import_path = r'D:\dev_data\re\hcad'
# export_file = import_path+"\\"+ "hcad_w_pro_last_names" + ".pkl"
# dfp0 = pd.read_csv(file1)
### Save the temporary result as pickle to save time in future
export_path = r'D:\dev_data\re\hcad'
path_to_file = export_path+"\\"+ "hcad_w_pro_first_last_names" + '.pkl'
dfo5.to_pickle(path_to_file, protocol=4)

## Import mddle processed work

In [ ]:
## import the marketing data
import_path = r'D:\dev_data\re\hcad'
path_to_file = import_path+"\\"+'hcad_w_pro_first_last_names.pkl'
dfo5 = pd.read_pickle(path_to_file)
dfo5.shape

In [ ]:
dfo5.head(2)

In [ ]:
dfo5.insert(0, 'batch_id', range(0, 0 + len(dfo5)))

In [ ]:
dfp1 = dfp0[['DOCK_NUM', 'DE_NAME']]
dfp1 = dfp1.applymap(lambda s:s.lower() if type(s) == str else s)
    

In [ ]:
dfo6 = dfo5[dfo5['batch_id']<=2000-1]

In [ ]:
dfo6.head(2)

In [ ]:
dfo6 = dfo6.drop(['l_name'], axis=1)

In [ ]:
dfo6.head(2)

## Add names from query list

In [ ]:
dfp1 = dfp0[['DOCK_NUM', 'DE_NAME']]
dfp1 = dfp1.applymap(lambda s:s.lower() if type(s) == str else s)
dfp1.head(2)

In [ ]:
# Combine both lists
dfo6 =dfo6.append(dfp1)
dfo6.fillna('NaN', inplace=True)
dfo6 = dfo6.applymap(str)

In [ ]:
dfo6.head(2)

In [ ]:
vectorizer = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word',
                                       stop_words='english', ngram_range=(1, 1), 
                                       norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=True)#,
                                      # max_df=1, min_df=1, max_features=None)
X = vectorizer.fit_transform(dfo6['MAILTO_cleaned'])
print(vectorizer.get_feature_names()[:20])

In [ ]:
# Create df with vectorized values, headings and original index
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index=dfo6.index)
tfidf_df.shape

In [ ]:
tfidf_df.shape

In [ ]:
t1 = time.time()
# Get pairwise similarities for the vectorized df
tfidf_dfj = pd.DataFrame(cosine_similarity(tfidf_df, dense_output=True))
t = time.time()-t1
print("SELFTIMED:", t)
tfidf_dfj.shape

In [ ]:
tfidf_dfj.head()

In [ ]:
l = dfo6['MAILTO_cleaned'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
sc = text_cleaner(s)
names = [str(x) for x in sc.split('||||') if x]
################
l = dfo6['ACCOUNT'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
accounts = [str(x) for x in s.split('||||') if x]
################
l = dfo6['DOCK_NUM'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
dock_num = [str(x) for x in s.split('||||') if x]
################
l = dfo6['DE_NAME'].tolist()
matching = [s for s in l if "|" in s]
s = '||||'.join(l).lower()
de_name = [str(x) for x in s.split('||||') if x]


In [ ]:
t1 = time.time()

t=[]

# Part 01:
for j,k in enumerate(tfidf_dfj.values):
    for n in range(len(k)):
        t.append([j,n,k[n]])

# Part 02:
qq=[]
for i in range(len(t)):
    if t[i][0]==t[i][1]:
        qq.append([t[i][0],t[i][1],0])
    else:
        qq.append(t[i])
qq[:5]

t = time.time()-t1
print("SELFTIMED:", t)

In [ ]:
from collections import defaultdict
u=defaultdict(list)

# Part 01:
for i in range(len(qq)):
    u[qq[i][0]].append(qq[i][2])
    
updated_df=pd.DataFrame(u)

# Part 02:
position_maxVal=[]
for i in range(len(updated_df)):
    position_maxVal.append(np.argmax(updated_df[i]))

t = time.time()-t1
print("SELFTIMED:", t)

In [ ]:
t1 = time.time()

sent_comp=[]

for j in position_maxVal: # list of highest similarity index positions
    # this creates in order our names w/ highest similiarity by row    
    sent_comp.append(names[j])
    sent_comp

# names based on highest similarity value per row as DF
similar_names=pd.DataFrame(sent_comp,columns=['Similar Names'])

# similiarity values rounded 4 decimal places finding max value per row
similarity_value_=pd.DataFrame(round(updated_df.max(axis=1),4), columns=['Similarity Value'])

In [ ]:
# Get accounts and names
p_names=pd.DataFrame(names,columns=['Names'])
p_accounts=pd.DataFrame(accounts,columns=['Accounts'])
p_dock_num=pd.DataFrame(dock_num,columns=['DockNum'])
p_de_name=pd.DataFrame(de_name,columns=['De_Name'])

# put everything together
cos_sim_df=pd.concat([p_accounts, p_names,similar_names,similarity_value_, p_dock_num, p_de_name], axis=1)


In [ ]:
cos_sim_df.head()

In [ ]:
cos_sim_df[~(cos_sim_df.DockNum.isnull()) ]

## Helper functions

In [ ]:
############################################# DO NOT DELETE ############################################# 

# Help functions to gather basic descriptions
def describe(df):
    return pd.concat([df.describe().T,
                      df.mad().rename('mean abs dev'),
                      df.skew().rename('skew'),
                      df.kurt().rename('kurt'),
                      df.nunique().rename('unique')
                     ], axis=1).T

def data_type_summary(df):
    # get Object data type summary
    df_stat_object = pd.DataFrame([])
    df_stat_object = df.describe(include = ['O'])
    df_stat_object.loc['dtype'] = df.dtypes
    df_stat_object.loc['size'] = len(df)
    #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    #df_stat_object.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    
    
    # get numerical data type summary
    df_stat_num = pd.DataFrame([])
    df_stat_num = df.describe(include = [np.number])
    df_stat_num.loc['dtype'] = df.dtypes
    df_stat_num.loc['size'] = len(df)
    #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    df_stat_object.loc['% null'] = ((df.isnull().sum()/df0.shape[0])*100).round(3)
    
    # # get date data type summary
    # df_stat_date = df.describe(include = ['datetime64']) 
    # df_stat_date.loc['dtype'] = df.dtypes
    # df_stat_date.loc['size'] = len(df)
    # #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    # #df_stat_object.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    
    result = pd.concat([df_stat_num, df_stat_object], axis=1, sort=False)
    return result

In [ ]:
############################################# DO NOT DELETE ############################################# 

def describe_with_nulls(df):
    nan_cols = [i for i in df.columns if df[i].isnull().any()]
    # get Object data type summary
    df_stat_num = pd.DataFrame([])
    #df_stat_num = df0[nan_cols].describe(include = [np.number, 'O', 'datetime64']) 
    df_stat_num = df0[nan_cols].describe(include = [np.number, 'O']) 
    
    df_stat_num.loc['dtype'] = df.dtypes
    df_stat_num.loc['size'] = len(df)
    #df_stat_num.loc['% null'] = df.isnull().count().round(2)
    df_stat_num.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    return df_stat_num

In [ ]:
############################################# DO NOT DELETE ############################################# 
# Function to move specific column to the left side for easier view
def move_to_left(df, column_name):
    df= df[ [str(column_name)] + [ col for col in df.columns if col != str(column_name) ] ]
    return df

def move_to_left_get_dup(df, column_name):
    df = df[ [str(column_name)] + [ col for col in df.columns if col != str(column_name) ] ]
    df = df[df.duplicated(str(column_name))]
    return df

In [ ]:
# Sample a subset of data
df1 = df0.sample(100000)
df1.shape

In [ ]:
df1.head(2)

In [ ]:
describe_with_nulls(df1)

## Pre-processing and feature engineering

In [ ]:
legal_cols = ['LEGAL_DSCR_1','LEGAL_DSCR_2','LEGAL_DSCR_3','LEGAL_DSCR_4']
date_cols = ['NOTICE_DATE','LAST_INSPECTED_DATE','NEW_OWNER_DATE']
num_cols = ['TOTAL_LAND_AREA', 'TOTAL_BUILDING_AREA', 'ACREAGE', 'LAND_VALUE',\
            'IMPROVEMENT_VALUE', 'ASSESSED_VALUE', 'NXT_BUILDING',\
            'TOTAL_APPRAISED_VALUE', 'TOTAL_MARKET_VALUE', 'PRIOR_LND_VALUE', 'YR_IMPR',\
            'PRIOR_IMPR_VALUE', 'PRIOR_TOTAL_APPRAISED_VALUE', 'PRIOR_TOTAL_MARKET_VALUE', 'TOTAL_RCN_VALUE']
# lower case
df1 = df1.apply(lambda x: x.astype(str).str.lower())
df1[num_cols] = df1[num_cols].apply(pd.to_numeric, errors='coerce') # numeric type
df1[num_cols] = df1[num_cols].fillna(0).astype(float) # important to fill Na before converting ot int
df1['SITE_ADDR_3']= df1['SITE_ADDR_3'].astype(str).str.zfill(5) # fix the zip code
df1['all_legal'] = df1[legal_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1) # Combine all legal
df1.drop(legal_cols, axis=1, inplace=True)
df1['UNDELIVERABLE'].replace("n", 0,inplace=True)
df1['UNDELIVERABLE'].replace("y", 1,inplace=True)
df1['PROTESTED'].replace("n", 0,inplace=True)
df1['PROTESTED'].replace("y", 1,inplace=True)

# Calculate $/sqf

df1['land_val_sqf'] = df1['LAND_VALUE']/df1['TOTAL_LAND_AREA']
df1['build_val_sqf'] = df1['IMPROVEMENT_VALUE']/df1['TOTAL_BUILDING_AREA']
df1['tot_appr_val_per_build_sqf'] = df1['TOTAL_APPRAISED_VALUE']/df1['TOTAL_BUILDING_AREA']
df1['tot_mark_val_per_build_sqf'] = df1['TOTAL_MARKET_VALUE']/df1['TOTAL_BUILDING_AREA']

# Filter away very low values
df1 = df1[df1['TOTAL_APPRAISED_VALUE'] >10000 ]
df1 = df1[df1['PRIOR_TOTAL_APPRAISED_VALUE'] >10000 ]
df1 = df1[df1['LAND_VALUE'] >10000 ]

# Add a unique column for concatination using when actual index cannot be used
df1['unique_id'] = np.arange(df1.shape[0])
move_to_left(df1, 'unique_id')
df1['ones'] = 1

In [ ]:
# Add sum by groupby for each neigborhood
df1['neighborhood_total'] = df1.groupby('NEIGHBORHOOD_CODE')['ones'].transform('sum')

In [ ]:
df1.shape

In [ ]:
data_type_summary(df1)

In [ ]:
# Calculate the ownership age
current_year = dt.datetime.today().year
df1['NEW_OWNER_YEAR'] = pd.to_numeric(df1['NEW_OWNER_YEAR'], errors='coerce').astype('Int64')
df1['new_owner_age']  = current_year - df1['NEW_OWNER_YEAR']

In [ ]:
# Function to tag as an absentee
def is_absentee (x):
    return (x['SITE_ADDR_1'][:5] != x['MAIL_ADDR_1'][:5])*1
# Apply the  
df1['SITE_ADDR_1'] = df1['SITE_ADDR_1'].astype(str)
df1['MAIL_ADDR_1'] = df1['MAIL_ADDR_1'].astype(str)
df1['absentee'] = df1.apply(is_absentee, axis=1)
df1.drop(['SITE_ADDR_1','MAIL_ADDR_1'], axis=1, inplace=True)

In [ ]:
# Create a dict for state class categories divided into residential and commercial
state_class_dict = {}
state_class_residential = ['a1', 'a2', 'a3', 'b1', 'b2']
state_class_commercial = set(df1['STATE_CLASS']) - set(state_class_residential)

state_class_vals = ['residential', 'commercial']
for key in set(df1['STATE_CLASS']):
    for value in state_class_vals:
        if key in state_class_residential:
            state_class_dict[key] = 'residential'
            #state_class_vals.remove(value) 
        else:
            state_class_dict[key] = 'commercial'


In [ ]:
# def map_state_codes(row, state_codes_dict):
#     return values_dict[row]
def map_state_codes(row, values_dict):
    return values_dict.get(row, 0)
    
#state_codes_dict = {'a1': 'residential', 'a2': 'residential','b1': 'residential', 'b2': 'residential','b1': 'residential'}
df1['STATE_CLASS_TYPE'] = df1['STATE_CLASS'].apply(map_state_codes, args = (state_class_dict,))

In [ ]:
df2 = df1.copy()

In [ ]:
df2.shape

In [ ]:
# Get object numbers before adding dummy 
df2_obj = df2.select_dtypes(include=['O'])
df2_obj.shape, df2_obj.columns

In [ ]:
# Categorical list to keep

cat_lst = [
       ##'MAIL_STATE', 'SITE_ADDR_2', 'SITE_ADDR_3', 'STATE_CLASS'
       ##,'SCHOOL_DIST', 'MAP_FACET', 
       ## 'KEY_MAP'
       'NEIGHBORHOOD_CODE'
       ##,'MARKET_AREA_1', 'MARKET_AREA_2', 'NXT_BUILDING', 'NOTICE_DATE_YEAR', 'UNDELIVERABLE',
       ##'LAST_INSPECTED_YEAR', 'LAST_INSPECTED_BY', 'NEW_OWNER_YEAR',
       ##'NEW_OWNER_MONTH','STATE_CLASS_TYPE'
       ]

In [ ]:
df2 = df2.drop(['all_legal'], axis=1)

# Get object numbers before adding dummy 
df2_obj = df2.select_dtypes(include=['O'])
df2_obj.shape, df2_obj.columns

In [ ]:
# Get all count of all unique values of categorical data type
pd.unique(df2[df2_obj.columns].values.ravel()).shape

In [ ]:
# List the scope of the dataset for a specific range of properties
df2 = df2[df2["STATE_CLASS_TYPE"]=='residential']
df2 = df2[df2["NXT_BUILDING"] ==1]
df2 = df2[df2["TOTAL_BUILDING_AREA"] <= 8000]
df2 = df2[df2["TOTAL_BUILDING_AREA"] > 0]
df2 = df2[df2["TOTAL_LAND_AREA"] <= 50000]
df2 = df2[df2["neighborhood_total"] >= 7]
df2 = df2[df2["TOTAL_MARKET_VALUE"] < 10000000]
df2 = df2[df2["tot_appr_val_per_build_sqf"] <=1000]
df2 = df2[df2["tot_mark_val_per_build_sqf"] <=1000]

In [ ]:
df2.shape

In [ ]:
col = ['TOTAL_BUILDING_AREA', 'TOTAL_LAND_AREA', 'LAND_VALUE', \
        'IMPROVEMENT_VALUE', 'TOTAL_APPRAISED_VALUE', \
        'TOTAL_MARKET_VALUE']

## Preliminary analysis of owners who protested using pair-plot

In [ ]:
sns.set()
with sns.plotting_context("notebook", font_scale=1.1):
    sns.pairplot(df2 \
        , vars=col, hue='PROTESTED')
plt.show()

## Value per sqf for building vs. land

In [ ]:
# prepare the data for plotting
# separate x and y
x = df2["TOTAL_BUILDING_AREA"]
y = df2["TOTAL_LAND_AREA"]

# instanciate the figure
fig = plt.figure(figsize = (20, 6))
# in this case we use gridspec.
gs = fig.add_gridspec(5, 5)
ax1 = fig.add_subplot(gs[:4, :-1])

# plot the data
# main axis: scatter plot
ax1.scatter(x, y) #, c = df4.target.astype('category').cat.codes) 

# set the labels for x and y
ax1.set_xlabel("Building Area, sqf")
ax1.set_ylabel("Land Area, sqf")

# set the title for the main plot
ax1.set_title("Building vs Land Area, sqf")

# prettify the plot
# get rid of some of the spines to make the plot nicer
ax1.spines["right"].set_color("None")
ax1.spines["top"].set_color("None")

# using familiar slicing, get the bottom axes and plot
ax2 = fig.add_subplot(gs[4:, :-1])
ax2.hist(x, 40, orientation = 'vertical', color = "pink")

# invert the axis (it looks up side down)
ax2.invert_yaxis()

# prettify the plot
# set the ticks to null
ax2.set_xticks([])
ax2.set_yticks([])
# no axis to make plot nicer
ax2.axison = False

# using familiar slicing, get the left axes and plot
ax3 = fig.add_subplot(gs[:4, -1])
ax3.hist(y, 40, orientation = "horizontal", color = "pink")

# prettify the plot
# set the ticks to null
ax3.set_xticks([])
ax3.set_yticks([])
# no axis to make plot nicer
ax3.axison = False

# make all the figures look nicier
fig.tight_layout()


## Focusing on areas with multiple properties

In [ ]:
df2_neighboorhood = df2[df2['neighborhood_total'] >= 30]
df2_neighboorhood = df2_neighboorhood.sort_values(by='neighborhood_total', ascending=False)
df2_neighboorhood.shape

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4), linewidth=5, facecolor='cyan')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
x = df2_neighboorhood['NEIGHBORHOOD_CODE']
y = df2_neighboorhood['neighborhood_total']
plt.xticks(rotation='vertical')
ax.bar(x, y)
ax.set(
    xlim=(0, 100), \
    ylim=(50, 180), \
    xlabel='Neighborhood Code', ylabel='Frequency' \
    , title='Histogram')
fig.suptitle('Neighborhood with multiple properties', size=20, y=1.03)

## Discrepancies in land value within same neighborhood

In [ ]:
x, y = df2_neighboorhood['NEIGHBORHOOD_CODE'], df2_neighboorhood['tot_appr_val_per_build_sqf']
c, s = df2_neighboorhood['land_val_sqf'], df2_neighboorhood['neighborhood_total']
 
fig, ax = plt.subplots(figsize=(20, 6))
scatter = ax.scatter(x, y, c=c \
    , s=s
    )

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                loc ="lower center", title="Mean Land $/sqf", fontsize='small', ncol=7) 
ax.add_artist(legend1)

# produce a legend with a cross section of sizes from the scatter
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
legend2 = ax.legend(handles, labels, loc="upper right", title="Size: No. Properties")

ax.set(xlim=(0, 100), ylim=(0, 180))
plt.xlabel(r'Neighborhood Code')
plt.xticks(rotation='vertical')
plt.ylabel(r'% Mean Total Appraised Build $/sqf')
plt.show()

## Discrepancies in building value within same neighborhood

In [ ]:
x, y = df2_neighboorhood['NEIGHBORHOOD_CODE'], df2_neighboorhood['land_val_sqf']
c, s = df2_neighboorhood['tot_appr_val_per_build_sqf'], df2_neighboorhood['neighborhood_total']
 
fig, ax = plt.subplots(figsize=(20, 6))
scatter = ax.scatter(x, y, c=c \
    , s=s
    )

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                loc ="upper center", title="Mean Build $/sqf", fontsize='small', ncol=4) 
ax.add_artist(legend1)

# produce a legend with a cross section of sizes from the scatter
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
legend2 = ax.legend(handles, labels, loc="upper right", title="Size: No. Properties")
ax.set(xlim=(0, 100), ylim=(0, 180))
plt.xlabel(r'Neighborhood Code')
plt.xticks(rotation='vertical')
plt.ylabel(r'% Mean Total Land Appraised $/sqf')
plt.show()

## Look at central tendencies using groupby

In [ ]:
table = pd.pivot_table(df2, index=['NEIGHBORHOOD_CODE'] \
    , values=['PROTESTED','TOTAL_MARKET_VALUE', 'TOTAL_APPRAISED_VALUE', 'PRIOR_TOTAL_APPRAISED_VALUE'\
    , 'TOTAL_BUILDING_AREA', 'TOTAL_LAND_AREA', 'LAND_VALUE' \
    ,'ones'], aggfunc=[np.sum,np.mean], fill_value=0)
table = table.reset_index()
table.columns = table.columns.map('_'.join)

In [ ]:
table.shape

In [ ]:
table = table.drop([
    'sum_PRIOR_TOTAL_APPRAISED_VALUE', 'sum_TOTAL_APPRAISED_VALUE' \
    ,'sum_TOTAL_BUILDING_AREA', 'sum_TOTAL_MARKET_VALUE', 'mean_PROTESTED'\
    ,'sum_TOTAL_BUILDING_AREA','sum_TOTAL_LAND_AREA'
    ,'mean_ones'], axis=1)
table.head()

In [ ]:
# Calculate some metrics
table['percent_protested'] = ((table['sum_PROTESTED'] / table['sum_ones']) * 100).round()
table['mean_market_excess_val'] = table['mean_TOTAL_MARKET_VALUE'] - table['mean_TOTAL_APPRAISED_VALUE']
table['mean_appraised_1yr_delta_val'] = table['mean_TOTAL_APPRAISED_VALUE'] - table['mean_PRIOR_TOTAL_APPRAISED_VALUE']
table['mean_appraised_per_sqf'] = table['mean_TOTAL_APPRAISED_VALUE'] / table['mean_TOTAL_BUILDING_AREA']
table['mean_land_val_per_sqf'] = table['mean_TOTAL_LAND_AREA'] / table['mean_TOTAL_BUILDING_AREA']
table = move_to_left(table, 'NEIGHBORHOOD_CODE_')
table = table[table['sum_ones'] >=10] # Focus on neighborhood with multiple properties
table = table.sort_values('mean_TOTAL_MARKET_VALUE', ascending=False)


In [ ]:
table.shape

In [ ]:
table.columns

In [ ]:
x, y = table['NEIGHBORHOOD_CODE_'], table['mean_TOTAL_MARKET_VALUE']
c, s = table['mean_market_excess_val'], table['percent_protested']
 
fig, ax = plt.subplots(figsize=(20, 4))
scatter = ax.scatter(x, y, c=c \
    , s=s
    )

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                loc="upper center", title="Mean Market Excess Value", fontsize='small', ncol=3)               
ax.add_artist(legend1)

# produce a legend with a cross section of sizes from the scatter
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
legend2 = ax.legend(handles, labels, loc="upper right", title="No. Protesting ")
ax.set(xlim=(0, 100), ylim=(10000, 1200000))
plt.xlabel(r'Neighborhood Code')
plt.xticks(rotation='vertical')
plt.ylabel(r'% Mean Total Market Value')
plt.show()

In [ ]:
data_type_summary(df2)

In [ ]:
df3 = df2.copy()

# Start working on Marketing Targe Group

In [ ]:
df_p.shape, df_o.shape

In [ ]:
df_p.head(2)

In [ ]:
df_p.columns

In [ ]:
df_p2 = df_p.copy()
# Drop all exepct focus columns
df_p2 = df_p2.loc[:, ['DOCK_NUM','DE_NAME', 'DE_FIRST_NAME', 'DE_MIDDLE_NAME', 'DE_LAST_NAME']]
df_p2['f_l_name'] = df_p2['DE_FIRST_NAME'] + ' ' + df_p2['DE_LAST_NAME']

In [ ]:
df_p2.head(2)

In [ ]:
df0.sample(2)

In [ ]:
df

## Add dummy variables for some categorical data 

In [ ]:
df3.dropna(inplace=True)

In [ ]:
df1.shape, df2.shape, df3.shape

In [ ]:
# Add categorical data of Neighborhood Code as binary
df3 = pd.get_dummies(data= df3, columns=["NEIGHBORHOOD_CODE"], prefix='neig_code')
df3.columns = df3.columns.str.replace(".", "_")

In [ ]:
# df2 = move_to_left(df2, 'NEIGHBORHOOD_CODE')

In [ ]:
df1.shape, df2.shape, df3.shape

In [ ]:
df3.head(1)

## Split the data into training, test and validation

### The label data is only for to validate the unsupervised result and is not used in the training

In [ ]:
# Select size=size from subset of a
np.random.seed(123)
indices = np.random.choice(a=15000, size=3000, replace=False)

# Y is the target variable
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

Y = df3['PROTESTED']
X = df3[df3.columns.difference(['absentee'])]
X_num = X.select_dtypes(include=numerics)
X_col_names = X_num.columns
X = X[X_col_names]
Y = Y.iloc[indices]
X = X.iloc[indices]

# numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# df7_num = df7.select_dtypes(include=numerics)
#X = df3.drop(['UNDELIVERABLE'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)
X_train, X_validation, y_train, y_validation   = train_test_split(X_train, y_train, test_size=0.25, random_state=465)

train_index = range(0,len(X_train))
validation_index = range(len(X_train),len(X_train)+len(X_validation))
test_index = range(len(X_train)+len(X_validation), len(X_train)+len(X_validation)+len(X_test))

print("df0.shape:{}, df1.shape:{}, df2.shape:{}, df3.shape:{}, X.shape:{}, Y.shape:{}, X_train.shape:{}, X_test.shape:{}, y_train.shape:{}, y_test.shape{}, y_val.shape{}".format(df0.shape, df1.shape, df2.shape, df3.shape, X.shape, Y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_validation.shape))

## Performance MetricsSilhouette Score
### Following we will use some of these metric to assess the clustering resutls:

Silhouette Score 
Tis measure doesn't need to know the ground truth and can be used to check, at the same time, the intra-cluster cohesion and the inter-cluster separation. 
This value is bounded between -1 and 1. A value close to -1 indicates that the average intra-cluster distance is greater than the average nearest-cluster index.
Viceversa, a value close to 1 indicates that the algorithm achieved a very good level of internal cohesion and inter-cluster separation.

Completeness score
This score is complementary to the Homogeneity score. Its purpose is to provide a piece of information about the assignment of samples belonging to the same class. 
Also, a good clustering algorithm should assign all samples with the same true label to the same cluster. 

Homogeneity score
This score is useful to check whether the clustering algorithm meets the requirement that a cluster should contain only samples belonging to a single class. 
It's bounded between 0 and 1, with low values indicating a low homogeneity. 

V-score
This score is the harmonic mean of precision and recall employed on clustering results when the ground truth is given.

The Adjusted Mutual Info
The main goal is of this score is to evaluate the level of agreement between Ytrue and Ypred without taking into account the permutations.
This value is equal to 0 in the case of the total absence of agreement and equal to 1 when Ytrue and Ypred completely agree. 

Adjusted Rand Index
This score is useful to compare the original label distribution with the clustering prediction. 
Ideally, we'd like to reproduce the exact ground truth distribution, but in general, this is very difficult in real-life scenarios. 
The RA measure is bounded between -1 and 1. A value close to -1 indicates a prevalence of wrong assignments, while a value close to 1 indicates that the clustering algorithm is correctly reproducing the ground truth distribution.





# Use of PSA with Kmeans

## Find the optimum components for the PCA

In [ ]:
time_start = time.time()
n_components = 2
whiten = False
random_state = 2020
pca2 = PCA(n_components=n_components, random_state=random_state)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
pca2 = PCA().fit(X_train_std)
print('Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
# time_start = time.time()
# print('Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:

fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(pca2.explained_variance_)
ax.set_xlabel('Components', fontsize=14)
ax.set_ylabel('Explained Variance', fontsize=14)
plt.show()

## Use PCA with optimal n-components

In [ ]:
time_start = time.time()
n_components = 2
whiten = False
random_state = 2020
pca = PCA(n_components=n_components, random_state=random_state)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_train_std_PCA = pca.fit_transform(X_train_std)
df_train_std_PCA = pd.DataFrame(X_train_std_PCA, index=X_train.index)# columns=['x', 'y'], 
dff = pd.concat([X_train, df_train_std_PCA], axis=1)

print('Time elapsed: {} seconds'.format(time.time()-time_start))

## # Compute the optimum inertia for kmeans

In [ ]:
inertias = []
init='k-means++'

for i in range(2, 70):
    km = KMeans(n_clusters=i, max_iter=1000, random_state=1000)
    km.fit(X_train_std_PCA )
    inertias.append(km.inertia_)

# Show the plot inertia vs. no. clusters
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(np.arange(2, 70, 1), inertias)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Inertia', fontsize=14)
ax.set_xticks(np.arange(2, 71, 2))
plt.show()

In [ ]:
# Check if any infinit or nan values may stop the fitting later
np.isnan(X_train_std_PCA.any()), np.isfinite(X_train_std_PCA.all()), #np.isnan(dff.any()), np.isfinite(dff.all())

In [ ]:
X_train_std_PCA.shape, X_train.shape, dff.shape

In [ ]:

time_start = time.time()
n_clusters = [2, 20, 50, 80]
mapping = [(0, 0), (0, 1), (1, 0), (1, 1)]

km_inertia = []
silhouette_sc = []
# Show the silhouette plots
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

for i, n in enumerate(n_clusters):
    km = KMeans(n_clusters=n, random_state=1000)
    y_pred = km.fit_predict(X_train_std_PCA)
    df_km = pd.DataFrame(y_pred, columns=['prediction'], index=X_train.index)
    dff = pd.concat([dff, df_km], axis=1)
    km_inertia.append((n, km.inertia_))
    silhouette_sc.append((n, silhouette_score(X_train_std_PCA, df_km['prediction'])))
   
    silhouette_values = silhouette_samples(X_train_std_PCA, df_km['prediction'])

    ax[mapping[i]].set_xticks([-0.15, 0.0, 0.25, 0.5, 0.75, 1.0])
    ax[mapping[i]].set_yticks([])
    ax[mapping[i]].set_title('%d clusters' % n)
    ax[mapping[i]].set_xlim([-0.15, 1])
    y_lower = 20

    for t in range(n):
        ct_values = silhouette_values[y_pred == t]
        ct_values.sort()

        y_upper = y_lower + ct_values.shape[0]

        color = cm.Accent(float(t) / n)
        ax[mapping[i]].fill_betweenx(np.arange(y_lower, y_upper), 0, ct_values, facecolor=color, edgecolor=color)

        y_lower = y_upper + 20

plt.show()

print('Time elapsed: {} seconds'.format(time.time()-time_start))

## Measure the performance of clusters 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
# Plotting Elbow Curve
x_iner = [x[0] for x in km_inertia]
y_iner  = [x[1] for x in km_inertia]
ax[0].plot(x_iner, y_iner)
ax[0].set_xlabel('Number of Clusters')
ax[0].set_ylabel('Intertia')
ax[0].set_title('Elbow Curve')
# Plotting Silhouetter Score
x_sil = [x[0] for x in silhouette_sc]
y_sil  = [x[1] for x in silhouette_sc]
ax[1].plot(x_sil, y_sil)
ax[1].set_xlabel('Number of Clusters')
ax[1].set_ylabel('Silhouetter Score')
ax[1].set_title('Silhouetter Score Curve')

## Pick optimal number of clusters and calcualtes all performance metrics  

In [ ]:
# Use PCA with optimal n-components
time_start = time.time()
km = KMeans(n_clusters=20, max_iter=1000, random_state=1000)
y_pred = km.fit_predict(X_train_std_PCA)
df_km = pd.DataFrame(y_pred, columns=['prediction'], index=X_train.index)
kmdff = pd.concat([X_train, df_km], axis=1)
print('Silhouette Score: {}'.format(silhouette_score(X_train_std_PCA, kmdff['prediction']).round(2)))
print('Completeness: {}'.format(completeness_score(kmdff['PROTESTED'], kmdff['prediction']).round(2)))
print('Homogeneity: {}'.format(homogeneity_score(kmdff['PROTESTED'], kmdff['prediction']).round(2)))
print('V-Score: {}'.format(v_measure_score(kmdff['PROTESTED'], kmdff['prediction']).round(2)))
print('Adj. Mutual info: {}'.format(adjusted_mutual_info_score(kmdff['PROTESTED'], kmdff['prediction']).round(2)))
print('Adj. Rand score: {}'.format(adjusted_rand_score(kmdff['PROTESTED'], kmdff['prediction'])))

print('Time elapsed: {} seconds'.format(time.time()-time_start))


## Performance
Based on Silhouette score we can conclude that this clustering is not helpful. The score is closer to zero than one indicating the algorithm achieved a very low level of internal cohesion and inter-cluster separation.
Similar conclusion is confirmed from other metrics.

# DBSCAN Custering

## Use t-SNE to reduce the dimentionality 

In [ ]:
from sklearn.manifold import TSNE
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

tsne = TSNE(n_components=2, perplexity=15, random_state=1000)
X_train_tsne = tsne.fit_transform(X_train_std)

df_tsne = pd.DataFrame(X_train_tsne, columns=['x', 'y'], index=X_train.index) 
dff1 = pd.concat([X_train, df_tsne], axis=1)

## t-SNE visualization: Is there any relationship between value/sqf and the property tax being protested or not

In [ ]:
sns.set()

fig, ax = plt.subplots(figsize=(18, 11))

with sns.plotting_context("notebook", font_scale=1.5):
    sns.scatterplot(x='x',
    y='y',
    size='build_val_sqf',
    sizes=(30, 400),
    style='PROTESTED',
    palette=sns.color_palette("tab10", 2),
    data=dff1,
    ax=ax)

ax.set_xlabel(r'$x$', fontsize=14)
ax.set_ylabel(r'$y$', fontsize=14)

plt.show()

## Use of DBSCAN

In [ ]:
dst = DBSCAN(eps=15, min_samples=3, metric='minkowski', p=2)
Y_pred_t = dst.fit_predict(X_train_tsne)
silhouette_score(dff1, Y_pred_t, metric='minkowski', p=2)

In [ ]:
from sklearn import metrics 

In [ ]:
# Find the best epsilon 
eps_grid = np.linspace(0.3, 1.2, num=10) 
silhouette_scores = [] 
eps_best = eps_grid[0] 
silhouette_score_max = -1 
model_best = None 
labels_best = None
AR_Index = []

In [ ]:
X_train_tsne_array = np.array(X_train_tsne)
for eps in eps_grid: 
    # Train DBSCAN clustering model 
    model = DBSCAN(eps=eps, min_samples=5).fit(X_train_tsne_array) 
 
    # Extract labels 
    labels = model.labels_
    
    # Extract performance metric  
    silhouette_score = round(metrics.silhouette_score(X_train_tsne_array, labels), 4) 
    silhouette_scores.append(silhouette_score) 
    print("Epsilon:", eps, " --> silhouette score:", silhouette_score) 
    
    # Store the best score and its associated epsilon value
    if silhouette_score > silhouette_score_max: 
        silhouette_score_max = silhouette_score 
        eps_best = eps 
        model_best = model 
        labels_best = labels

In [ ]:
# Plot silhouette scores vs epsilon 
plt.figure() 
plt.bar(eps_grid, silhouette_scores, width=0.05, color='k', align='center') 
plt.title('Silhouette score vs epsilon') 
 
# Best params 
print("Best epsilon =", eps_best) 

In [ ]:
# Store the associated model and labels for best epsilon 
model = model_best  
labels = labels_best

# Check for unassigned datapoints in the labels 
offset = 0 
if -1 in labels: 
    offset = 1 
    
# Number of clusters in the data  
num_clusters = len(set(labels)) - offset  
 
print("Estimated number of clusters =", num_clusters)

In [ ]:
# Extracts the core samples from the trained model 
mask_core = np.zeros(labels.shape, dtype=np.bool) 
mask_core[model.core_sample_indices_] = True 

In [ ]:
# Plot resultant clusters  
from itertools import cycle 
plt.figure() 
labels_uniq = set(labels) 
markers = cycle('vo^s<>') 

In [ ]:
for cur_label, marker in zip(labels_uniq, markers): 
    # Use black dots for unassigned datapoints 
    if cur_label == -1: 
        marker = '.' 
 
    # Create mask for the current label 
    cur_mask = (labels == cur_label) 
 
    cur_data = X_train_tsne_array[cur_mask & mask_core] 
    plt.scatter(cur_data[:, 0], cur_data[:, 1], marker=marker, 
             edgecolors='black', s=96, facecolors='none') 
    cur_data = X_train_tsne_array[cur_mask & ~mask_core] 
    plt.scatter(cur_data[:, 0], cur_data[:, 1], marker=marker, 
             edgecolors='black', s=32) 

plt.title('Data separated into clusters') 
plt.show()

## Performance
Based on Silhouette score we can conclude that this clustering is not helpful. The score is closer to zero than one indicating the algorithm achieved a very low level of internal cohesion and inter-cluster separation. This is also confirmed Adjusted Rand Index.

# Hierarchical Clustering 

In [ ]:
def plot_clustered_dataset(X, Y):
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    ax.grid()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    markers = ['o', 'd', '^', 'x', '1', '2', '3', 's']
    colors = ['r', 'b', 'g', 'c', 'm', 'k', 'y', '#cccfff']

    for i in range(X.shape[0]):
        ax.scatter(X[i, 0], X[i, 1], marker=markers[Y[i]], color=colors[Y[i]])

    plt.show()

# For reproducibility
np.random.seed(1000)

nb_samples = 3000

## Visualize the data

In [ ]:
# Show the dataset
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

ax.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')

ax.scatter(X_train_std_PCA[:, 0], X_train_std_PCA[:, 1], marker='o', color='b')
plt.show()

In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
# Compute the distance matrix
Xdist = pdist(X_train_std_PCA, metric='euclidean')

# Compute the linkage
Xl = linkage(Xdist, method='ward')

# Compute and show the dendrogram
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
Xd = dendrogram(Xl)
plt.show()


### Pick a proper number of clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score

ac = AgglomerativeClustering(n_clusters=7, linkage='complete')
Y_pred = ac.fit_predict(X_train_std_PCA)

print('Silhouette score (Complete): %.3f' % silhouette_score(X_train_std_PCA, Y_pred))
print('Adjusted Rand score (Complete): %.3f' % adjusted_rand_score(y_train, Y_pred))

fig, ax = plt.subplots(1, 1, figsize=(10, 8))

ax.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')

markers = ['o', 'd', '^', 'x', '1', '2', '3', 's']
colors = ['r', 'b', 'g', 'c', 'm', 'k', 'y', '#cccfff']
col_labels = list(set(ac.labels_))

plt.scatter(X_train_std_PCA[:, 0], X_train_std_PCA[:, 1], c=ac.labels_)#, marker=markers[y_train[i]], color=colors[y_train[i]])
plt.show()

## Performance
Based on Silhouette score we can conclude that this clustering is not helpful. However it is much better than previous algorithms. 
The Adjusted Rand Index is also very low indicating a poor comparability between the original label distribution and the clustering prediction.

# Best model selection
Hierarchical clustering performed best compared with other algorithms given the Silhouette score. However Adjusted Rand Index is low for all three which make this clustering project in need of further fine tuining and possibly adding more data.